# AQI Predictor - Exploratory Data Analysis

This notebook provides comprehensive exploratory data analysis for the Air Quality Index prediction project.

## Contents
1. Data Loading & Overview
2. Missing Data Analysis
3. Statistical Summary
4. Time Series Trends
5. Correlation Analysis
6. Feature Distributions
7. Seasonal Patterns
8. Key Insights

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# Import project modules
import sys
sys.path.append('..')
from src.config import settings
from src.data_fetch import fetch_air_quality
from src.feature_engineering import build_features
from src.feature_store import FeatureStore

## 1. Data Loading & Overview

In [ ]:
# Fetch fresh data for analysis
cities = list(settings.city_coords.keys())
print(f"Available cities: {cities}")

# Fetch data for default city
city = settings.default_city
raw_data = fetch_air_quality(city, past_days=2, forecast_days=3)
print(f"\nFetched {len(raw_data)} hourly records for {city}")
print(f"Date range: {raw_data['time'].min()} to {raw_data['time'].max()}")

In [ ]:
# Display data structure
print("Column types:")
print(raw_data.dtypes)
print(f"\nShape: {raw_data.shape}")
raw_data.head(10)

## 2. Missing Data Analysis

In [ ]:
# Check for missing values
missing = raw_data.isnull().sum()
missing_pct = (missing / len(raw_data)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct.round(2)
}).sort_values('Missing Count', ascending=False)

print("Missing Values Analysis:")
display(missing_df[missing_df['Missing Count'] > 0])

# Visualize missing data
if missing.sum() > 0:
    fig, ax = plt.subplots(figsize=(10, 5))
    missing_df[missing_df['Missing Count'] > 0]['Missing %'].plot(kind='barh', ax=ax, color='coral')
    ax.set_xlabel('Missing Percentage (%)')
    ax.set_title('Missing Data by Column')
    plt.tight_layout()
    plt.show()
else:
    print("\n✅ No missing values in the dataset!")

## 3. Statistical Summary

In [ ]:
# Numerical columns summary
numeric_cols = raw_data.select_dtypes(include=[np.number]).columns.tolist()
print("Statistical Summary of Numeric Features:")
raw_data[numeric_cols].describe().round(2)

In [ ]:
# AQI-specific statistics
if 'us_aqi' in raw_data.columns:
    aqi_col = 'us_aqi'
elif 'european_aqi' in raw_data.columns:
    aqi_col = 'european_aqi'
else:
    aqi_col = None

if aqi_col:
    aqi_data = raw_data[aqi_col].dropna()
    print(f"\nAQI ({aqi_col}) Statistics:")
    print(f"  Mean: {aqi_data.mean():.2f}")
    print(f"  Median: {aqi_data.median():.2f}")
    print(f"  Std Dev: {aqi_data.std():.2f}")
    print(f"  Min: {aqi_data.min():.2f}")
    print(f"  Max: {aqi_data.max():.2f}")
    print(f"  Skewness: {aqi_data.skew():.2f}")
    print(f"  Kurtosis: {aqi_data.kurtosis():.2f}")

## 4. Time Series Trends

In [ ]:
# Plot AQI over time
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# AQI trend
if aqi_col and aqi_col in raw_data.columns:
    axes[0, 0].plot(raw_data['time'], raw_data[aqi_col], color='purple', linewidth=1.5)
    axes[0, 0].set_title(f'{aqi_col.upper()} Over Time')
    axes[0, 0].set_xlabel('Time')
    axes[0, 0].set_ylabel('AQI')
    axes[0, 0].axhline(y=100, color='orange', linestyle='--', label='Moderate Threshold')
    axes[0, 0].axhline(y=150, color='red', linestyle='--', label='Unhealthy Threshold')
    axes[0, 0].legend()

# PM2.5 trend
if 'pm2_5' in raw_data.columns:
    axes[0, 1].plot(raw_data['time'], raw_data['pm2_5'], color='red', linewidth=1.5)
    axes[0, 1].set_title('PM2.5 Concentration Over Time')
    axes[0, 1].set_xlabel('Time')
    axes[0, 1].set_ylabel('PM2.5 (μg/m³)')

# PM10 trend
if 'pm10' in raw_data.columns:
    axes[1, 0].plot(raw_data['time'], raw_data['pm10'], color='blue', linewidth=1.5)
    axes[1, 0].set_title('PM10 Concentration Over Time')
    axes[1, 0].set_xlabel('Time')
    axes[1, 0].set_ylabel('PM10 (μg/m³)')

# Temperature trend
if 'temperature_2m' in raw_data.columns:
    axes[1, 1].plot(raw_data['time'], raw_data['temperature_2m'], color='orange', linewidth=1.5)
    axes[1, 1].set_title('Temperature Over Time')
    axes[1, 1].set_xlabel('Time')
    axes[1, 1].set_ylabel('Temperature (°C)')

plt.tight_layout()
plt.savefig('../reports/time_series_trends.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Correlation Analysis

In [ ]:
# Build features for correlation analysis
features_df = build_features(raw_data)
print(f"Feature-engineered dataset shape: {features_df.shape}")
print(f"Features: {list(features_df.columns)}")

In [ ]:
# Correlation matrix
numeric_features = features_df.select_dtypes(include=[np.number])
corr_matrix = numeric_features.corr()

# Plot correlation heatmap
fig, ax = plt.subplots(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', 
            center=0, square=True, linewidths=0.5, ax=ax,
            annot_kws={'size': 8})
ax.set_title('Feature Correlation Heatmap', fontsize=14)
plt.tight_layout()
plt.savefig('../reports/correlation_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Top correlations with target
if 'target_aqi' in features_df.columns:
    target_corr = corr_matrix['target_aqi'].drop('target_aqi').abs().sort_values(ascending=False)
    print("Top Features Correlated with Target AQI:")
    print(target_corr.head(10).round(3))
    
    # Visualize top correlations
    fig, ax = plt.subplots(figsize=(10, 6))
    target_corr.head(10).plot(kind='barh', ax=ax, color='steelblue')
    ax.set_xlabel('Absolute Correlation')
    ax.set_title('Top 10 Features Correlated with Target AQI')
    ax.invert_yaxis()
    plt.tight_layout()
    plt.savefig('../reports/target_correlations.png', dpi=150, bbox_inches='tight')
    plt.show()

## 6. Feature Distributions

In [ ]:
# Distribution of key features
key_cols = ['pm2_5', 'pm10', 'us_aqi', 'european_aqi', 'temperature_2m', 'relative_humidity_2m']
available_cols = [c for c in key_cols if c in raw_data.columns]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(available_cols[:6]):
    data = raw_data[col].dropna()
    axes[i].hist(data, bins=30, color='steelblue', edgecolor='white', alpha=0.7)
    axes[i].axvline(data.mean(), color='red', linestyle='--', label=f'Mean: {data.mean():.1f}')
    axes[i].axvline(data.median(), color='orange', linestyle='--', label=f'Median: {data.median():.1f}')
    axes[i].set_title(f'{col} Distribution')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequency')
    axes[i].legend(fontsize=8)

# Hide unused subplots
for j in range(len(available_cols), 6):
    axes[j].set_visible(False)

plt.tight_layout()
plt.savefig('../reports/feature_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Box plots for outlier detection
fig, axes = plt.subplots(1, len(available_cols[:4]), figsize=(14, 5))
for i, col in enumerate(available_cols[:4]):
    raw_data.boxplot(column=col, ax=axes[i])
    axes[i].set_title(f'{col}')

plt.suptitle('Box Plots for Outlier Detection', y=1.02)
plt.tight_layout()
plt.savefig('../reports/box_plots.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Seasonal/Temporal Patterns

In [ ]:
# Add time features for analysis
raw_data['hour'] = raw_data['time'].dt.hour
raw_data['dayofweek'] = raw_data['time'].dt.dayofweek
raw_data['day_name'] = raw_data['time'].dt.day_name()

# Hourly pattern
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# AQI by hour
if aqi_col and aqi_col in raw_data.columns:
    hourly_aqi = raw_data.groupby('hour')[aqi_col].mean()
    axes[0].bar(hourly_aqi.index, hourly_aqi.values, color='steelblue', alpha=0.8)
    axes[0].set_xlabel('Hour of Day')
    axes[0].set_ylabel('Average AQI')
    axes[0].set_title('Average AQI by Hour of Day')
    axes[0].set_xticks(range(0, 24, 2))

# PM2.5 by hour
if 'pm2_5' in raw_data.columns:
    hourly_pm25 = raw_data.groupby('hour')['pm2_5'].mean()
    axes[1].bar(hourly_pm25.index, hourly_pm25.values, color='coral', alpha=0.8)
    axes[1].set_xlabel('Hour of Day')
    axes[1].set_ylabel('Average PM2.5')
    axes[1].set_title('Average PM2.5 by Hour of Day')
    axes[1].set_xticks(range(0, 24, 2))

plt.tight_layout()
plt.savefig('../reports/hourly_patterns.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Scatter plots: Pollutants vs Weather
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

if 'pm2_5' in raw_data.columns and 'temperature_2m' in raw_data.columns:
    axes[0].scatter(raw_data['temperature_2m'], raw_data['pm2_5'], alpha=0.5, c='red')
    axes[0].set_xlabel('Temperature (°C)')
    axes[0].set_ylabel('PM2.5')
    axes[0].set_title('PM2.5 vs Temperature')

if 'pm2_5' in raw_data.columns and 'relative_humidity_2m' in raw_data.columns:
    axes[1].scatter(raw_data['relative_humidity_2m'], raw_data['pm2_5'], alpha=0.5, c='blue')
    axes[1].set_xlabel('Humidity (%)')
    axes[1].set_ylabel('PM2.5')
    axes[1].set_title('PM2.5 vs Humidity')

if 'pm2_5' in raw_data.columns and 'wind_speed_10m' in raw_data.columns:
    axes[2].scatter(raw_data['wind_speed_10m'], raw_data['pm2_5'], alpha=0.5, c='green')
    axes[2].set_xlabel('Wind Speed (m/s)')
    axes[2].set_ylabel('PM2.5')
    axes[2].set_title('PM2.5 vs Wind Speed')

plt.tight_layout()
plt.savefig('../reports/scatter_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Key Insights & Conclusions

In [ ]:
# Summary insights
print("="*60)
print("KEY INSIGHTS FROM EXPLORATORY DATA ANALYSIS")
print("="*60)

print(f"\n📊 DATASET OVERVIEW:")
print(f"   • Total records: {len(raw_data)}")
print(f"   • Time span: {raw_data['time'].max() - raw_data['time'].min()}")
print(f"   • Features available: {len(raw_data.columns)}")

if 'pm2_5' in raw_data.columns:
    pm25 = raw_data['pm2_5'].dropna()
    print(f"\n🔴 PM2.5 ANALYSIS:")
    print(f"   • Average: {pm25.mean():.1f} μg/m³")
    print(f"   • Range: {pm25.min():.1f} - {pm25.max():.1f} μg/m³")
    print(f"   • Std Dev: {pm25.std():.1f}")

if aqi_col and aqi_col in raw_data.columns:
    aqi = raw_data[aqi_col].dropna()
    hazardous_pct = (aqi >= 100).sum() / len(aqi) * 100
    print(f"\n⚠️ AQI ANALYSIS:")
    print(f"   • Average AQI: {aqi.mean():.1f}")
    print(f"   • Moderate+ hours: {hazardous_pct:.1f}%")

print(f"\n📈 FEATURE CORRELATIONS:")
if 'target_aqi' in features_df.columns:
    top3 = target_corr.head(3)
    for feat, corr in top3.items():
        print(f"   • {feat}: {corr:.3f}")

print(f"\n✅ DATA QUALITY:")
missing_total = raw_data.isnull().sum().sum()
print(f"   • Missing values: {missing_total}")
print(f"   • Data completeness: {(1 - missing_total/(raw_data.shape[0]*raw_data.shape[1]))*100:.1f}%")

print("\n" + "="*60)

---

## Next Steps

Based on this EDA, recommended actions:

1. **Feature Selection**: Use highly correlated features for modeling
2. **Data Cleaning**: Handle any missing values detected
3. **Feature Engineering**: Create interaction terms for correlated features
4. **Model Training**: Start with tree-based models due to non-linear patterns

---
*Generated by AQI Predictor EDA Pipeline*